In [ ]:
# importing necessary libraries

import wget # a python library to download files from a given url
from mnist import MNIST #a python package which has the mnis
import os #library corresponding to os based operations
from PIL import Image as im #used to convert the given image array to a b/w image
import numpy as np 
import random
import statistics
from tqdm import tqdm #for timing
import matplotlib.pyplot as plt #for plots
import sklearn

## Code to download the train and test datasets from the given MNIST url

In [ ]:
#Run this cell to download the mnist dataset if it hasn't been downloaded already

print("Do you want to download the mnist dataset ?(y/n)")
choice = str(input())
if(choice.lower() == 'y'): #if choice is yes, proceed with the downloads   
    master_dir = os.getcwd() #this is the directory we're working in
    mnist_dir = "mnist" #dir to store the downloaded mnist dataset
    if(os.path.isdir(os.path.join(master_dir,mnist_dir))=='True'): #if the directory doesn't exist
            os.mkdir(os.path.join(master_dir,mnist_dir))#make a directory called mnist in the master_dir

    #defining list datasets which has strings pertaining to the necessary extensions on the mnist database which gives us the files

    datasets = ["train-images-idx3-ubyte.gz","train-labels-idx1-ubyte.gz","t10k-images-idx3-ubyte.gz","t10k-labels-idx1-ubyte.gz"]
    mnist_url = "http://yann.lecun.com/exdb/mnist/" #url containing the dataset

    for data in datasets:
        wget.download(os.path.join(mnist_url,data),out=mnist_dir)#downloads all the files to the dir called mnist


In [ ]:
# This cell is to unzip and format the downloaded mnist dataset
master_dir = os.getcwd() #this is the directory we're working in
mnist_dir = "mnist" #dir to store the downloaded mnist dataset
mnist_path = os.path.join(master_dir,mnist_dir) #change name to path where mnist is downloaded to

#creating an mnist object

mnist_obj = MNIST(mnist_path)
mnist_obj.gz = True #as I downloaded the .gz zip files
train_images,train_labels = mnist_obj.load_training() #loads the training dataset
test_images,test_labels   = mnist_obj.load_testing() #loads the testing dataset

print("Done extracting the data")    

In [ ]:
#converting the resulting labels into a one_hot_vector
N_classes = 10 #number of classes is 10 for mnist
one_hot_train = []
one_hot_test = []

for i,label in enumerate(train_labels):
    dummy = np.zeros(N_classes)
    dummy[label] = 1    #wherever the label is, make that index 1
    one_hot_train.append(dummy)
    
for i,label in enumerate(test_labels):
    dummy = np.zeros(N_classes)
    dummy[label] = 1    #wherever the label is, make that index 1
    one_hot_test.append(dummy)

In [ ]:
train_images = np.asarray(train_images)
np.transpose(train_images).shape
np.asarray(one_hot_train).shape
print(np.transpose(train_images))

### Images are of type list each image is a 784 long list
### Labels are of type int , which we convert to one hot of length 10
### About train: 60,000 images 
### About test:10,000 images


In [ ]:
#visualizing the image just for clarity
image_sample = np.asarray(train_images[0],dtype = np.uint8).reshape(28,28)

#creating image object
image_obj = im.fromarray(image_sample) #converts the array into a b/w image 28x28

#saving the image
image_obj.save('sample_digit.png')
print('Sample image has been saved')
image_obj.show()


In [ ]:
#modifications to training_data and testing_data

train_data = np.transpose(train_images)/255
one_hot_train = np.transpose(np.asarray(one_hot_train))

test_data = np.transpose(test_images)/255
one_hot_test = np.transpose(np.asarray(one_hot_test))

print(test_data.shape,one_hot_test.shape)
print(train_data.shape,one_hot_train.shape)


In [ ]:
test_data = np.transpose(test_images)
one_hot_test = np.transpose(np.asarray(one_hot_test))

print(test_data.shape,one_hot_test.shape)


In [ ]:
# Library of all the activation functions
#input should be an integer array/matrix or an integer

def sigmoid(x):  #input is x and output is the sigmoid of x 1/(1+exp(-x)) 
    return 1/(1+np.exp(-x))

def ReLU(x): #input is x and output is ReLU(x),max{x,0} 
    #val = 1e-9*np.ones_like(x) #regularization
    return x*(x > 0) # cool way to compute ReLU for vectors and scalars alike 

def tanh(x): #input is x and output is tanh(x) = (exp(x)-exp(-x))/(exp(x)+exp(-x))
    return np.tanh(x)

def linear(x): #for closure input is x, output is x
    return x

def softmax(x): #input is a matrix x, output is the softmax of each element of the matrix,column-wise, ie softmax(xi) = exp(xi)/sum(exp(xj) over all j)
    #print(max(x))
    exp_x = np.exp(x)
    return exp_x/exp_x.sum(axis = 0) #summing along axis = 0,ie the columns of x

#create a dictionary hosting the function names

activation_library = {
    1:sigmoid,
    2:ReLU,
    3:tanh,
    4:linear,
    5:softmax
}


In [ ]:
# library of all the derivatives of the activation functions
#input should be an integer array or an integer

def sigmoid_derivative(x): # computes the derivative of the activation function sigmoid for input x
    return sigmoid(x)*(1-sigmoid(x))

def ReLU_derivative(x): #computes the derivative of the activation function ReLU for input x
    #val = 1e-9*np.ones_like(x) #regularization
    return 1*(x > 0)

def tanh_derivative(x): #computes the derivative of the activation function tanh for input x
    return 1- tanh(x)**2

def linear_derivative(x): #computes the derivative of the linear activation function for input x
    return 1

def softmax_derivative_for_CE(p_true,q_pred): #computes the derivative for softmax activation at the final layer of the MLP
    return q_pred - p_true

    

activation_derivative_library = {
    1: sigmoid_derivative,
    2: ReLU_derivative,
    3: tanh_derivative,
    4: linear_derivative,
    5: softmax_derivative_for_CE
}


In [ ]:
#learning rates
def constant_learning_rate(learning_rate,i=0,thresh=9):
    return learning_rate

def exp_learning_rate(learning_rate,i=0,thresh=4): 
    if(i > thresh):
        return learning_rate*np.exp(-i)
    else:
        return learning_rate
    




learning_rate_library = {
    1:constant_learning_rate,
    2:exp_learning_rate
}

In [ ]:
#gradient descent
#perform one iteration of gradient descent

def basic_grad_descent(learning_rate,theta,minibatch_size,theta_grad,t,r_t=0,s_t=0,delta = 10e-10,pho_1=0.9,pho_2=0.85,bias = True,reg1 = False,reg1_lambda = 1e-3):
    if(bias == True):
        theta_new = theta - learning_rate*theta_grad.sum(axis = 1).reshape(theta.shape)/minibatch_size
        #print(theta.shape,theta_grad.shape,theta_new.shape)
    else:
        theta_new = theta - learning_rate*theta_grad/minibatch_size - np.sign(theta)*reg1*reg1_lambda/minibatch_size
    
    return theta_new,r_t,s_t

def momentum_gd(learning_rate,theta,minibatch_size,theta_grad,t,r_t = 0,s_t=0,delta = 10e-10,pho_1 = 0.9,pho_2=0.85,bias = True,reg1 = False,reg1_lambda = 1e-3): #can be converted to nesterov gd by taking derivative at theta_t - gamma*Vt-1
    if(bias == True):
        r_upd = pho_1*r_t + learning_rate*theta_grad.sum(axis = 1).reshape(theta.shape)/minibatch_size
    else:
        r_upd = pho_1*r_t + learning_rate*theta_grad/minibatch_size - np.sign(theta)*reg1*reg1_lambda/minibatch_size
        
    theta_new = theta - r_upd
    return theta_new,r_upd,s_t

def Adagrad(learning_rate,theta,minibatch_size,theta_grad,t,r_t = 0,s_t=0,delta = 10e-10,pho_1=0.9,pho_2=0.85,bias = True,reg1 = False,reg1_lambda = 1e-3):
    if(bias == True):
        r_upd = r_t + np.square(theta_grad).sum(axis = 1).reshape(theta.shape)/minibatch_size
        theta_new = theta - learning_rate*np.reciprocal((delta + np.sqrt(r_upd)))*theta_grad.sum(axis = 1).reshape(theta.shape)/minibatch_size
        
    else:
        r_upd = r_t + np.square(theta_grad)/minibatch_size + reg1*(reg1_lambda/minibatch_size)**2
        theta_new = theta - learning_rate*np.reciprocal((delta + np.sqrt(r_upd)))*theta_grad/minibatch_size - np.sign(theta)*reg1*reg1_lambda/minibatch_size
        
    return theta_new,r_upd,s_t

def RMS_prop(learning_rate,theta,minibatch_size,theta_grad,t,r_t=0,s_t=0,delta = 10e-10,pho_1 = 0.9,pho_2=0.85,bias = True,reg1 = False,reg1_lambda = 1e-3):
    if(bias == True):
        r_upd = pho_1*r_t + (1-pho_1)*np.square(theta_grad).sum(axis = 1).reshape(theta.shape)/minibatch_size
        theta_new = theta - learning_rate*np.reciprocal((delta + np.sqrt(r_upd)))*theta_grad.sum(axis = 1).reshape(theta.shape)/minibatch_size
        
    else:
        r_upd = pho_1*r_t + (1-pho_1)*np.square(theta_grad)/minibatch_size + reg1*(reg1_lambda/minibatch_size)**2
        theta_new = theta - learning_rate*np.reciprocal((delta + np.sqrt(r_upd)))*theta_grad/minibatch_size - np.sign(theta)*reg1*reg1_lambda/minibatch_size
        
    return theta_new,r_upd,s_t

def ADAM(learning_rate,theta,minibatch_size,theta_grad,t,r_t = 0,s_t = 0,delta = 10e-10,pho_1 = 0.9,pho_2 = 0.85,bias = True):
    if(bias == True):
        s_upd = pho_1*s_t + (1-pho_1)*np.square(theta_grad).sum(axis = 1).reshape(theta.shape)/minibatch_size
        r_upd = pho_2*r_t + (1-pho_2)*np.square(theta_grad).sum(axis = 1).reshape(theta.shape)/minibatch_size
        
    else:
        s_upd = pho_1*s_t + (1-pho_1)*np.square(theta_grad)/minibatch_size
        r_upd = pho_2*r_t + (1-pho_2)*np.square(theta_grad)/minibatch_size
        
    if(t == 0):
        s_val = 0
        r_val = 0
    else:
        s_val = s_upd/(1-pho_1**t)
        r_val = r_upd/(1-pho_2**t)
    
    theta_new = theta - learning_rate*np.multiply(1/(delta + np.sqrt(r_val)),s_val)

    return theta_new,r_upd,s_upd
        
    
    
    
grad_descent_library = {
    1:basic_grad_descent,
    2:momentum_gd,
    3:Adagrad,
    4:RMS_prop,
    5:ADAM
}

In [ ]:
#loss functions

def cross_entropy_loss(p,q): #p,q are vectors, p is the true label, q is our prediction, function returns cross entropy loss: -sum(pi*log(qi))
        epsilon = 1e-11 #epsilon regularization should qi be very small
        return -np.mean((p*np.log(q+epsilon)).sum(0))
    
def l2_loss(p,q,weights,batch_size,lambda_reg = 8e-4 ):
    reg_loss = 0
    for i in weights.keys():
        reg_loss += np.sum(np.square(weights[i]))
    
    return cross_entropy_loss(p,q) + (lambda_reg/(2*batch_size))*reg_loss  
 
def l1_loss(p,q,weights,batch_size,lambda_reg = 1e-1):
    reg_loss = 0
    for i in weights.keys():
        reg_loss += np.sum(np.abs(weights[i]))
    
    return cross_entropy_loss(p,q) + (lambda_reg/batch_size)*reg_loss

reg_loss = {
    1:l1_loss,
    2:l2_loss
}
    


In [ ]:
# score normalization 
def basic_stats(y_pred,y_true):
    total_labels = len(y_true[0])
    correct_pred = ((np.argmax(y_pred,axis = 0) == np.argmax(y_true,axis = 0))*np.ones(total_labels)).sum() #sum gives no of correct pred
    accuracy     = correct_pred/total_labels
    true_labels  = np.asarray(list(enumerate(np.argmax(y_true,axis = 0))))
    y_vals       = y_pred[true_labels[:,1],true_labels[:,0]]
    mean_error   = np.mean(1-y_vals)
    stdev_error  = statistics.stdev(1-y_vals)
    return accuracy,mean_error,stdev_error

def more_stats(y_pred,y_true):
    confusion_mat = np.dot(y_true,y_pred.T)
    precision        = np.diag(confusion_mat)/np.sum(confusion_mat,axis=0)
    recall           = np.diag(confusion_mat)/np.sum(confusion_mat,axis=1)
    f1_score         = 2*precision*recall/(precision+recall)
    return confusion_mat,precision,recall,f1_score
    



In [ ]:
# defining a class MLP and also the accompanying functions required to operate the MLP
class MLP: #class named multi layer perceptron
    
    def __init__(self,architecture,activations,train_images,train_labels,test_images,test_labels,plot= True,plot_mode = 0,threshold = 1e-5,minibatch_size=64,n_epochs=15,learning_rate_choice=1,learning_rate=0.025,grad_descent_choice = 1,regularization = 0,reg_lambda =8e-4, noise = True,noise_stdev = 0.25e-2): #class constructor which initializes all the parameters required to fire the MLP
            self.architecture     = architecture  #list containing the number of neurons in each layer so here it'll be [h1,h2,h3]
            self.no_layers        = len(architecture)+1 #not considering the input layer
            self.activations      = activations   #activation function for each layer
            self.train_data       = train_images
            self.train_labels     = train_labels
            self.minibatch_size   = minibatch_size
            self.n_epochs         = n_epochs
            self.lr_choice        = learning_rate_choice
            self.learning_rate    = learning_rate
            self.grad_descent     = grad_descent_choice
            self.input_dim        = len(train_images)
            self.train_nos        = len(train_images[0])
            self.output_dim       = len(train_labels)
            self.test_data        = test_images
            self.test_labels      = test_labels
            self.test_nos         = len(test_images[0])
            self.weights          = {}
            self.biases           = {}
            self.inactive_neurons = {}
            self.thresh           = threshold
            self.test_pred        = []
            self.train_losses     = []
            self.test_losses      = []
            self.train_accuracy   = []
            self.test_accuracy    = [] 
            self.plot             = plot
            self.plot_mode        = plot_mode
            self.reg              = regularization
            self.reg_lambda       = reg_lambda
            self.noise            = noise
            self.noise_stdev      = noise_stdev
            
            
            
            for i in range(self.no_layers): #initialising the dictionaries
                self.weights[i]          = []
                self.biases[i]           = []
                self.inactive_neurons[i] = []
                
    def glorot_initialization(self,fan_in,fan_out):
        dl = np.sqrt(6/(fan_in+fan_out))
        return np.asarray(np.random.uniform(-dl,dl,(fan_out,fan_in)),dtype = np.float64)
    
    def initialize_weights_biases(self):
        
        for i in range(self.no_layers):
            
            if(i==0):
                self.weights[i] = self.glorot_initialization(self.input_dim,self.architecture[0])
                self.biases[i]  = np.zeros((self.architecture[0],1),dtype = np.float64)
                #print(self.weights[i].shape,self.biases[i].shape)
            
            elif(i== len(self.architecture)):
                self.weights[i] = self.glorot_initialization(self.architecture[-1],self.output_dim)
                self.biases[i]  = np.zeros((self.output_dim,1),dtype = np.float64)
                #print(self.weights[i].shape,self.biases[i].shape)
            
            else:
                self.weights[i] = self.glorot_initialization(self.architecture[i-1],self.architecture[i])
                self.biases[i]  = np.zeros((self.architecture[i],1),dtype = np.float64)
                #print(self.weights[i].shape,self.biases[i].shape)
                
    
    def forward_pass(self,data): #data is a matrix with dimensions: input_dim x minibatch_size
        z_vals = {}
        a_vals = {}
        q_vals = np.zeros((self.output_dim,len(data)))
        for i in range(self.no_layers):
            if(i == 0):
                #print(self.biases[0].shape,self.weights[0].shape,data.shape)
                z_vals[i] = np.dot(self.weights[0],data) + np.dot(self.biases[0],np.ones((1,len(data[0])))) #broadcasting results in an index error for last iter of epoch so do it explicitly
                #print(z_vals[i].shape)
                a_vals[i] = activation_library[self.activations[i]](z_vals[i])
                #print("zvals shape,avals shape",i,":",z_vals[i].shape,a_vals[i].shape)
            
            else:
                z_vals[i] = np.dot(self.weights[i],a_vals[i-1]) + np.dot(self.biases[i],np.ones((1,len(data[0]))))
                a_vals[i] = activation_library[self.activations[i]](z_vals[i])
                #print("zvals shape,avals shape",i,":",z_vals[i].shape,a_vals[i].shape)
            
            if(i == len(self.architecture)): #output values
                #q_vals = softmax(a_vals[i])
                q_vals = activation_library[self.activations[-1]](a_vals[i]) #final activation 
                #print(q_vals.shape)
        return z_vals,a_vals,q_vals
    
    
    def backprop(self,z_vals,a_vals,q_vals,data_labels,data):
        deltas = {}
        weight_gradients = {}
        bias_gradients = {}
        
        for i in range(len(self.architecture),-1,-1): #performs the backprop across all layers starting from the final layer
            
            if(i == len(self.architecture)):
                deltas[i] = activation_derivative_library[self.activations[-1]](data_labels,q_vals)   #delta index starts from 4 instead of 5 in class
            else:
                deltas[i] = np.multiply(np.dot(np.transpose(self.weights[i+1]),deltas[i+1]),activation_derivative_library[self.activations[i]](z_vals[i])) #element wise product np.multiply
            
            self.inactive_neurons[i].append(np.sum((deltas[i] < self.thresh)*1)/(deltas[i].shape[0]*deltas[i].shape[1]))    
            if(i==0):
                #print(deltas[i].shape,np.transpose(data).shape)
                weight_gradients[i] = np.dot(deltas[i],np.transpose(data))
            else:
                #print(deltas[i].shape,np.transpose(a_vals[i-1]).shape)
                weight_gradients[i] = np.dot(deltas[i],np.transpose(a_vals[i-1]))
                
            bias_gradients[i] = deltas[i]
        
        return weight_gradients,bias_gradients
    
        
    
    def train_MLP(self):
        N_batches = int(len(self.train_data[0])//self.minibatch_size + 1)
        self.initialize_weights_biases()
        if(self.reg == 2):
            const = 1-(self.reg_lambda/self.minibatch_size)
            reg1 = False
        else:
            if(self.reg == 1):
                reg1 = True
            else:
                reg1 = False
            const      = 1 #constant
        train_loss = []
        test_loss  = []
        iterations = []
        r_w          = [0]*self.no_layers
        s_w          = [0]*self.no_layers
        r_b          = [0]*self.no_layers
        s_b          = [0]*self.no_layers
    
        for e in range(self.n_epochs):
            print("Epoch",e,"is training")
            #shuffling the dataset for every epoch
            train_data_epoch,train_labels_epoch = sklearn.utils.shuffle(self.train_data.T,self.train_labels.T)
            train_data_epoch   = train_data_epoch.T
            if(self.noise == True):
                train_data_epoch += np.random.normal(0, self.noise_stdev, np.shape(train_data_epoch))
            train_labels_epoch = train_labels_epoch.T
            
            for n in tqdm(range(N_batches)):
            #for n in range(N_batches):
                if(n == N_batches-1):
                    #data        = self.train_data[:,n*self.minibatch_size:]
                    #data_labels = self.train_labels[:,n*self.minibatch_size:]
                    data        = train_data_epoch[:,n*self.minibatch_size:]
                    data_labels = train_labels_epoch[:,n*self.minibatch_size:]
                
                else:
                    data        = train_data_epoch[:,n*self.minibatch_size:(n+1)*self.minibatch_size]
                    data_labels = train_labels_epoch[:,n*self.minibatch_size:(n+1)*self.minibatch_size]
                    
                
                
                z_vals,a_vals,q_vals = self.forward_pass(data)
                
                weight_gradients,bias_gradients = self.backprop(z_vals,a_vals,q_vals,data_labels,data)
                
                for i in range(self.no_layers):
                    # (learning_rate,theta,minibatch_size,theta_grad,t,r_t = 0,s_t = 0,delta = 10e-10,pho_1 = 0.9,pho_2 = 0.85,bias = True)
                    if((e == 0) and (n == 0)):    
                        self.weights[i],r_w[i],s_w[i] = grad_descent_library[self.grad_descent](learning_rate_library[self.lr_choice](self.learning_rate,e),const*self.weights[i],self.minibatch_size,weight_gradients[i],e*N_batches+n,bias = False,reg1 = reg1)
                        self.biases[i],r_b[i],s_b[i]  = grad_descent_library[self.grad_descent](learning_rate_library[self.lr_choice](self.learning_rate,e),self.biases[i],self.minibatch_size,bias_gradients[i],e*N_batches+n)  
                        r_w[i] = np.asarray(r_w[i])
                        r_b[i] = np.asarray(r_b[i])
                        s_w[i] = np.asarray(s_w[i])
                        s_b[i] = np.asarray(s_b[i])
                    else:
                        self.weights[i],r_w[i],s_w[i] = grad_descent_library[self.grad_descent](learning_rate_library[self.lr_choice](self.learning_rate,e),const*self.weights[i],self.minibatch_size,weight_gradients[i],e*N_batches+n,r_w[i],s_w[i],bias = False,reg1 = reg1)
                        self.biases[i],r_b[i],s_b[i]  = grad_descent_library[self.grad_descent](learning_rate_library[self.lr_choice](self.learning_rate,e),self.biases[i],self.minibatch_size,bias_gradients[i],e*N_batches+n,r_b[i],s_b[i])                                       
                        r_w[i] = np.asarray(r_w[i])
                        r_b[i] = np.asarray(r_b[i])
                        s_w[i] = np.asarray(s_w[i])
                        s_b[i] = np.asarray(s_b[i])
                
                if(self.reg):
                    train_loss.append(reg_loss[self.reg](data_labels,q_vals,self.weights,self.minibatch_size))
                else:
                    train_loss.append(cross_entropy_loss(data_labels,q_vals))
                
                if(((N_batches*e+n)%200 == 0)):#every 200 iterations
                    q_test = self.forward_pass(self.test_data)[-1]
                    #print(q_test.shape)
                    test_loss.append(cross_entropy_loss(self.test_labels,q_test))
                    iterations.append(N_batches*e+n)             
                   
        self.test_losses  = test_loss           
        self.train_losses = train_loss
        self.test_pred    = self.forward_pass(self.test_data)[-1]
        
        if(self.plot):
            self.basic_plots(iterations,self.plot_mode)
            
        
        #return test_loss,train_loss
    
    
    def basic_plots(self,iter_list,Mode = 0):
        
        #first plotting the training and the testing losses
        fig, (ax1,ax2) =plt.subplots(2,sharex = True)
        
        ax1.plot(self.train_losses,'o-')
        ax1.set_title("Average Training Losses vs iterations")
        ax1.grid(True)
        ax1.set_ylabel("Training loss")
        
        ax2.plot(iter_list,self.test_losses,'+-')
        ax2.set_title("Average Testing Losses vs iterations")
        ax2.grid(True)
        ax2.set_ylabel("Testing loss")
        ax2.set_xlabel("<- Iterations ->")
        
        fig.set_figwidth(10)
        fig.set_figheight(8)
        fig.show()
        accuracy,mean,std = basic_stats(self.test_pred,self.test_labels)
        print("Accuracy:",accuracy)
        print("Mean of errors:",mean)
        print("Standard deviation of errors:",std)
        if(Mode != 0):
            
            #plotting the percentage of inactive neurons
            fig,ax = plt.subplots()
            for i in range(self.no_layers):
                ax.plot(self.inactive_neurons[i],label = "Neurons at layer"+str(i+1))

            plt.grid(True)
            plt.legend(loc = 'best')
            plt.xlabel('<-Iterations ->')
            plt.ylabel('% of Inactive neurons')
            fig.suptitle('Percentage of Inactive neurons present in every layer')
            plt.show()

            #plotting the confusion_matrix for both train and test
            #cm_1,p_1,r_1,f1_1 = more_stats(self.train_pred,self.train_labels)
            cm_2,p_2,r_2,f1_2 = more_stats(self.test_pred,self.test_labels)
            
            '''
            plt.imshow(cm_1, interpolation='nearest', cmap=plt.cm.Wistia)
            classNames = ['0','1','2','3','4','5','6','7','8','9']
            plt.title('Train Data: Confusion Matrix')
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            tick_marks = np.arange(len(classNames))
            plt.xticks(tick_marks, classNames)
            plt.yticks(tick_marks, classNames)

            for i in range(len(classNames)):
                for j in range(len(classNames)):
                    plt.text(j,i,str(cm_1[i][j]))
            plt.show()
            '''
                   
            plt.imshow(cm_2, interpolation='nearest', cmap=plt.cm.Wistia)
            plt.colorbar()
            classNames = ['0','1','2','3','4','5','6','7','8','9']
            plt.title('Test Data: Confusion Matrix')
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            tick_marks = np.arange(len(classNames))
            plt.xticks(tick_marks, classNames)
            plt.yticks(tick_marks, classNames)
            
            #for i in range(len(classNames)):
            #    for j in range(len(classNames)):
            #        plt.text(j,i,str(cm_2[i][j]))
            plt.show()
            
            '''       
            #precision, recall plots
            fig, ax = plt.subplots()
            index = np.arange(n_groups)
            bar_width = 0.25
            opacity = 0.8

            rects1 = plt.bar(index, p_1, bar_width,
            alpha=opacity,
            color='b',
            label='Precision')

            rects2 = plt.bar(index + bar_width, r_1, bar_width,
            alpha=opacity,
            color='g',
            label='Recall')

            rects3 = plt.bar(index + 2*bar_width,f1_1,bar_width,alpha=opacity,color = 'r',label="F1 Score")

            plt.xlabel('Digits')
            plt.ylabel('Scores')
            plt.title('Train Data: Precision, Recall and F1 score')
            plt.xticks(index + 1.5*bar_width, classNames)
            plt.legend()

            plt.tight_layout()
            plt.show()
            '''

            fig, ax = plt.subplots()
            index = np.arange(len(p_2))
            bar_width = 0.25
            opacity = 0.8

            rects1 = plt.bar(index, p_2, bar_width,
            alpha=opacity,
            color='b',
            label='Precision')

            rects2 = plt.bar(index + bar_width, r_2, bar_width,
            alpha=opacity,
            color='g',
            label='Recall')

            rects3 = plt.bar(index + 2*bar_width,f1_2,bar_width,alpha=opacity,color = 'r',label="F1 Score")

            plt.xlabel('Digits')
            plt.ylabel('Scores')
            plt.title('Test Data: Precision, Recall and F1 score')
            plt.xticks(index + 1.5*bar_width, classNames)
            plt.legend()

            plt.tight_layout()
            plt.show()


        
                
                    
                
        
            
        
        
                
            
        
                
                
            
            
            


In [ ]:
#main script for the assignment

#question 1
#experimenting with learning rates
grid_of_learning_rates = np.asarray([0.01,0.02,0.025,0.03,0.04,0.05])

for learning_rate in tqdm(grid_of_learning_rates):
    NN_sigmoid = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=0,learning_rate = learning_rate,noise= False)
    NN_sigmoid.train_MLP()

In [ ]:
grid_of_learning_rates = np.asarray([0.06,0.07,0.08,0.09,0.1])
for learning_rate in tqdm(grid_of_learning_rates):
    NN_sigmoid = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=0,learning_rate = learning_rate,noise= False)
    NN_sigmoid.train_MLP()

In [ ]:
#experimenting with gradient descent algorithms

NN_sigmoid = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=0,learning_rate_choice = 2,learning_rate = 0.025,noise= False)
NN_sigmoid.train_MLP()

for i in range(4):
    NN_sigmoid = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=0,learning_rate_choice = 1,learning_rate = 0.025,grad_descent_choice = i+1,noise= False)
    NN_sigmoid.train_MLP()
    

In [ ]:
#comparing the various activation functions

NN_sigmoid = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=1,learning_rate_choice = 1,learning_rate = 0.025,grad_descent_choice = 4,noise= False)
NN_sigmoid.train_MLP()

NN_relu   = MLP([500,250,100],[2,2,2,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=1,learning_rate_choice = 1,learning_rate = 0.025,grad_descent_choice = 1,noise= False)
NN_relu.train_MLP()

NN_tanh  = MLP([500,250,100],[3,3,3,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=1,learning_rate_choice = 1,learning_rate = 0.025,grad_descent_choice = 4,noise= False)
NN_tanh.train_MLP()

In [ ]:
# regularization with a very high value
NN_sigmoid_r1 = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=1,n_epochs=15,learning_rate_choice = 1,learning_rate = 0.025,grad_descent_choice = 4,regularization=1,noise= False)
NN_sigmoid_r1.train_MLP()


In [ ]:
# regularization and augmentation
NN_sigmoid_r1 = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=1,learning_rate_choice = 1,learning_rate = 0.025,grad_descent_choice = 4,regularization=1,noise= False)
NN_sigmoid_r1.train_MLP()

NN_sigmoid_r2 = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=1,learning_rate_choice = 1,learning_rate = 0.025,grad_descent_choice = 4,regularization=2,noise= False)
NN_sigmoid_r2.train_MLP()

NN_sigmoid_n = MLP([500,250,100],[1,1,1,4,5],train_data,one_hot_train,test_data,one_hot_test,plot_mode=1,learning_rate_choice = 1,learning_rate = 0.025,grad_descent_choice = 4)
NN_sigmoid_n.train_MLP()

In [ ]:
plt.imshow(NN_sigmoid_r1.weights[2], interpolation='nearest', cmap=plt.cm.Wistia)
plt.colorbar()
plt.title('Weights for L1 regularization with high lambda')
plt.ylabel('Second Hidden Layer')
plt.xlabel('Third Hidden layer')
plt.show()


plt.imshow(NN_sigmoid_r2.weights[2], interpolation='nearest', cmap=plt.cm.Wistia)
plt.colorbar()
plt.title('Weights for L2 regularization')
plt.ylabel('Second Hidden Layer')
plt.xlabel('Third Hidden layer')
plt.show()

plt.imshow(NN_sigmoid_n.weights[2], interpolation='nearest', cmap=plt.cm.Wistia)
plt.colorbar()
plt.title('Weights for noise addition')
plt.ylabel('Second Hidden Layer')
plt.xlabel('Third Hidden layer')
plt.show()


print(np.sum((NN_sigmoid.weights[2] > NN_sigmoid_r2.weights[2] )*1)/(NN_sigmoid_r1.weights[2].shape[0]*NN_sigmoid_r1.weights[2].shape[1]))
print(np.sum((NN_sigmoid.weights[2] > NN_sigmoid_n.weights[2] )*1)/(NN_sigmoid_r1.weights[2].shape[0]*NN_sigmoid_r1.weights[2].shape[1]))

In [ ]:
# Feature extraction part, run in collab
import cv2
from sklearn import svm
from sklearn.metrics import classification_report
from keras.datasets import mnist
import numpy as np
from google.colab.patches import cv2_imshow
from sklearn.neighbors import KNeighborsClassifier as knn

def deskew(img, imgSize):
    # calculate image moments
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        # no deskewing needed
        return img.copy()

    # calculate skew based on central moments
    skew = m['mu11'] / m['mu02']

    # calculate affine transformation to correct skewness
    M = np.float32([[1, skew, -0.5*imgSize*skew], [0, 1, 0]])

    # apply affine transformation
    img = cv2.warpAffine(img, M, (imgSize, imgSize), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)

    return img

# Load the mnist dataset
(trainX, trainY), (testX, testY) = mnist.load_data()


imsize = 28 # size of image (28x28)

# HOG parameters:
winSize = (imsize, imsize) # 28, 28
blockSize = (imsize//2, imsize//2) # 14, 14    
cellSize = (imsize//2, imsize//2) #14, 14
blockStride = (imsize//4, imsize//4) # 7, 7
nbins = 9
signedGradients = True
derivAperture = 1
winSigma = -1.0
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64

# define the HOG descriptor
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma, 
                        histogramNormType, L2HysThreshold, gammaCorrection, nlevels, signedGradients)

# compute HOG descriptors
train_values = []
for i in range(trainX.shape[0]):
    trainX[i] = deskew(trainX[i], 28) # deskew the current image
    train_hog = hog.compute(trainX[i]) # compute the HOG features
    train_values.append(train_hog) # append it to the train values list

test_values = []
for i in range(testX.shape[0]):
    testX[i] = deskew(testX[i], 28) # deskew the current image
    test_hog = hog.compute(testX[i]) # compute the HOG features
    test_values.append(test_hog) # append it to the test values list


train_values = np.resize(train_values, (trainX.shape[0], 81))


test_values = np.resize(test_values, (testX.shape[0], 81))

# classifier
clf = svm.SVC(C=1.0, kernel='rbf') #try linear also, same output is obtained 98.5 accuracy
clf2 = knn(n_neighbors=200) #about 96 accuracy
clf2.fit(train_values,trainY)
clf.fit(train_values, trainY)

# print the classification report
print(classification_report(testY, clf.predict(test_values)))
print(classification_report(testY,clf2.predict(test_values)))


In [ ]:
NN_relu = MLP([500,250,100],[2,2,2,4,5],np.asarray(train_values).T,one_hot_train,np.asarray(test_values).T,one_hot_test,plot_mode=1,n_epochs = 25,learning_rate = 0.0025,grad_descent_choice = 4)
NN_relu.train_MLP()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
trainX = trainX.reshape(5000,784)#if you're using 5000 samples
testX  = testX.reshape(5000,784)#if you're using 5000 samples
scaler1.fit(trainX)
train = scaler1.transform(trainX)
scaler1.fit(testX)
test  = scaler1.transform(testX)

pca1 = PCA(n_components = 81)
pca1.fit(train)
train_pca = pca1.transform(train)
pca1.fit(test)
test_pca = pca1.transform(test)

print(test_pca.shape,train_pca.shape)

clf = svm.SVC(C=1.0, kernel='rbf')
clf2 = knn(n_neighbors=30)
clf2.fit(train_pca,trainY)
clf.fit(train_pca, trainY)

# print the classification report
print(classification_report(testY, clf.predict(test_pca)))
print(classification_report(testY,clf2.predict(test_pca)))